<a href="https://colab.research.google.com/github/whatmakeart/ffmpeg-datamosh/blob/main/ffmpeg_datamosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Inspired and adapted from Reddit thread comments https://www.reddit.com/r/datamoshing/comments/t46x3i/datamoshing_with_ffmpeg_howto_in_comments/
# Converted to Colab with plain language instructions in ChatGPT io-preview

# Import necessary libraries
from google.colab import files
import os
import subprocess
import shutil
import glob

# Step 1: Upload video files
print("Please upload your video files (you can upload a single assembled video or multiple clips):")
uploaded = files.upload()
filenames = list(uploaded.keys())

# Step 2: For each file, ask if the user wants to trim it
trim_options = {}
for filename in filenames:
    print(f"\nDo you want to trim {filename}? (y/n)")
    answer = input().lower()
    if answer == 'y':
        print(f"Enter start time in seconds (or leave blank for start of video):")
        start_time = input()
        print(f"Enter end time in seconds (or leave blank for end of video):")
        end_time = input()
        trim_options[filename] = {'start_time': start_time, 'end_time': end_time}
    else:
        trim_options[filename] = None

# Function to trim videos
def trim_video(input_file, output_file, start_time=None, end_time=None):
    cmd = ['ffmpeg', '-y']
    if start_time:
        cmd.extend(['-ss', start_time])
    if end_time:
        cmd.extend(['-to', end_time])
    cmd.extend(['-i', input_file, '-c', 'copy', output_file])
    subprocess.run(cmd)

# Step 3: Trim videos if needed
trimmed_files = []
for filename in filenames:
    if trim_options[filename]:
        start_time = trim_options[filename]['start_time'] if trim_options[filename]['start_time'] else None
        end_time = trim_options[filename]['end_time'] if trim_options[filename]['end_time'] else None
        output_file = f"trimmed_{filename}"
        trim_video(filename, output_file, start_time, end_time)
        trimmed_files.append(output_file)
    else:
        trimmed_files.append(filename)

# Step 4: Assemble trimmed files into one video
print("\nAssembling clips into one video...")
# Create input.txt file for ffmpeg concat
with open('input.txt', 'w') as f:
    for file in trimmed_files:
        f.write(f"file '{file}'\n")

# Run ffmpeg concat
cmd = ['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'input.txt', '-c', 'copy', 'original_video.mp4']
subprocess.run(cmd)

# Step 5: Convert to libxvid format with specific settings
print("\nDo you want to interpolate to a higher frame rate and scale the video? (y/n)")
interpolate_answer = input().lower()
if interpolate_answer == 'y':
    print("Enter desired frame rate (e.g., 60):")
    fps = input()
    print("Enter scale factor (e.g., 2 for 2x):")
    scale_factor = input()
    cmd = [
        'ffmpeg', '-y', '-i', 'original_video.mp4',
        '-vf', f'minterpolate=fps={fps}:mi_mode=mci,scale={scale_factor}*iw:{scale_factor}*ih',
        '-vcodec', 'libxvid',
        '-q:v', '1',
        '-g', '1000',
        '-qmin', '1',
        '-qmax', '1',
        '-flags', '+qpel+mv4',
        '-an',
        'xvid_video.avi'
    ]
else:
    cmd = [
        'ffmpeg', '-y', '-i', 'original_video.mp4',
        '-vcodec', 'libxvid',
        '-q:v', '1',
        '-g', '1000',
        '-qmin', '1',
        '-qmax', '1',
        '-flags', '+qpel+mv4',
        '-an',
        'xvid_video.avi'
    ]
print("Converting to xvid_video.avi...")
subprocess.run(cmd)

# Step 6: Create frames and save directories
os.makedirs('frames/save', exist_ok=True)

# Step 7: Extract raw frames as .raw files
print("Extracting raw frames as .raw files...")
cmd = [
    'ffmpeg', '-y', '-i', 'xvid_video.avi',
    '-vcodec', 'copy',
    '-start_number', '0',
    'frames/f_%04d.raw'
]
subprocess.run(cmd)

# Step 8: Extract images of all frames (optional for visual inspection)
os.makedirs('images', exist_ok=True)
print("Extracting images of all frames to images folder...")
cmd = [
    'ffmpeg', '-y', '-i', 'xvid_video.avi',
    '-start_number', '0',
    'images/i_%04d.jpg'
]
subprocess.run(cmd)

# Step 9: Identify I-frame numbers
print("Identifying I-frame numbers...")
cmd = [
    'ffprobe', '-v', 'error',
    '-select_streams', 'v:0',
    '-show_entries', 'frame=pict_type',
    '-of', 'csv',
    'xvid_video.avi'
]
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
pict_types = result.stdout.strip().split('\n')

iframe_numbers = []
for idx, line in enumerate(pict_types):
    if line.strip() == 'frame,I':
        iframe_numbers.append(idx + 1)  # Frame numbers start at 1
print(f"I-frame numbers (excluding the first frame): {iframe_numbers[1:]}\n")

# Step 10: Remove I-frames (except the first one) and replace with next frame
print("Removing I-frames and replacing them with the next frame to maintain frame count...")
frames_dir = 'frames'
raw_files = sorted(glob.glob(os.path.join(frames_dir, 'f_*.raw')))
# Map frame numbers to filenames
frame_file_dict = {}
for filename in raw_files:
    basename = os.path.basename(filename)
    frame_number = int(basename[2:6]) + 1  # Frame numbers start at 1
    frame_file_dict[frame_number] = filename

for iframe_number in iframe_numbers[1:]:  # Exclude the first frame
    current_frame_file = frame_file_dict.get(iframe_number)
    next_frame_file = frame_file_dict.get(iframe_number + 1)
    if current_frame_file and next_frame_file:
        # Move the current frame to frames/save
        dst = os.path.join(frames_dir, 'save', os.path.basename(current_frame_file))
        shutil.move(current_frame_file, dst)
        # Copy the next frame to replace the current frame
        shutil.copy(next_frame_file, current_frame_file)

# Step 11: Concatenate raw frames back into edited_video.avi
print("Concatenating raw frames to create edited_video.avi...")
sorted_raw_files = [frame_file_dict[i+1] for i in range(len(frame_file_dict))]
with open('edited_video.avi', 'wb') as outfile:
    for filename in sorted_raw_files:
        with open(filename, 'rb') as infile:
            shutil.copyfileobj(infile, outfile)

# Step 12: Convert edited_video.avi to final_video.mp4 and add back audio
print("Creating final_video.mp4 by adding back audio and scaling if necessary...")

print("\nDo you want to scale the final video to specific dimensions? (y/n)")
scale_answer = input().lower()
if scale_answer == 'y':
    print("Enter desired width (e.g., 1280):")
    width = input()
    scale_option = f'scale={width}:-2'
else:
    scale_option = 'scale=trunc(iw/2)*2:trunc(ih/2)*2'  # Ensure dimensions are even numbers

cmd = [
    'ffmpeg', '-y', '-i', 'edited_video.avi',
    '-i', 'original_video.mp4',
    '-vf', scale_option,
    '-map', '0:v:0',
    '-map', '1:a:0',
    '-vcodec', 'h264',
    '-shortest',
    'final_video.mp4'
]
subprocess.run(cmd)

# Step 13: Provide final_video.mp4 for download
print("\nProcessing complete! Downloading the final video...")
files.download('final_video.mp4')